In [16]:
import pandas as pd

In [17]:
# Read cleaned and create node label list
df = pd.read_csv("all_data.csv")

names = df[['name']]
names.to_csv('names.csv')

In [18]:
# Create a dataframe with all possible edges
edges = []
for i, row in df.iterrows():
    for j, row2 in df.iterrows():
        if i != j:
            edges.append([i, j])
        else:
            continue

edges_df = pd.DataFrame(edges, columns=['source', 'target'])
edges_df['weight'] = 0.0
edges_df

,source,target,weight
0,0,1,0.0
1,0,2,0.0
2,0,3,0.0
3,0,4,0.0
4,0,5,0.0
...,...,...,...
865,29,24,0.0
866,29,25,0.0
867,29,26,0.0
868,29,27,0.0


In [19]:
# Function to check for matches with an equal match
def check_for_matches_equal(df, column_name):
    matches = []
    for index, row in df.iterrows():
        matches.extend([(index, other_index) for other_index, other_row in df.iterrows() \
                    if other_row[column_name] == row[column_name] and other_index != index])
    return matches

In [20]:
# Function to check for matches with a greather than difference
def check_for_matches_great(df, column, x):
    matches = []
    for index, row in df.iterrows():
        matches.extend([(index, other_index) for other_index, other_row in df.iterrows() \
                        if (other_row[column] - row[column])**2 > x**2  and other_index != index])
    return matches

In [21]:
# Function to check for matches with a lesser than difference
def check_for_matches_less(df, column, x):
    matches = []
    for index, row in df.iterrows():
        matches.extend([(index, other_index) for other_index, other_row in df.iterrows() \
                        if (other_row[column] - row[column])**2 < x  and other_index != index])
    return matches

In [22]:
# Check for country matches
country_df = check_for_matches_equal(df, 'country')
country_df = pd.DataFrame(country_df, columns=['source', 'target'])
country_df

,source,target
0,0,1
1,0,2
2,0,3
3,0,4
4,0,5
...,...,...
65,21,23
66,22,21
67,22,23
68,23,21


In [23]:
# Check for clique matches
clique_df = check_for_matches_equal(df, 'clique')
clique_df = pd.DataFrame(clique_df, columns=['source', 'target'])
clique_df

,source,target
0,0,1
1,0,3
2,0,4
3,0,9
4,0,15
...,...,...
123,29,10
124,29,12
125,29,17
126,29,22


In [24]:
# Check for age gap over 25 year matches
age_df = check_for_matches_great(df, 'age', 25)
age_df = pd.DataFrame(age_df, columns=['source', 'target'])
age_df

,source,target
0,0,11
1,0,23
2,1,23
3,2,11
4,2,22
...,...,...
135,29,11
136,29,20
137,29,21
138,29,22


In [25]:
# Check for big 5 similarities
openness = check_for_matches_great(df, 'Openness', 1)
agreeableness = check_for_matches_great(df, 'Agreeableness', 1)
conscientiousness = check_for_matches_great(df, 'Conscientiousness', 1)
extraversion = check_for_matches_great(df, 'Extraversion', 1)
neuroticism = check_for_matches_great(df, 'Neuroticism', 1)

big5_df = pd.DataFrame(openness + agreeableness + conscientiousness + extraversion + neuroticism, columns=['source', 'target'])
big5_df

,source,target
0,0,1
1,0,2
2,0,5
3,0,6
4,0,7
...,...,...
1529,29,5
1530,29,18
1531,29,20
1532,29,23


In [26]:
# Adding weights for country matches
for index, row_short in country_df.iterrows():
    for index, row_long in edges_df.iterrows():
        if (row_short[['source', 'target']] == row_long[['source', 'target']]).all():
            edges_df.at[index, 'weight'] += 1

# Adding weights for clique matches
for index, row_short in clique_df.iterrows():
    for index, row_long in edges_df.iterrows():
        if (row_short[['source', 'target']] == row_long[['source', 'target']]).all():
            edges_df.at[index, 'weight'] += 1
        
# Substracting weights for age gaps
for index, row_short in age_df.iterrows():
    for index, row_long in edges_df.iterrows():
        if (row_short[['source', 'target']] == row_long[['source', 'target']]).all():
            edges_df.at[index, 'weight'] -= 1



In [29]:
# Adding weights for big 5
for index, row_short in big5_df.iterrows():
    for index, row_long in edges_df.iterrows():
        if (row_short[['source', 'target']] == row_long[['source', 'target']]).all():
            edges_df.at[index, 'weight'] += 0.2

In [28]:
edges_df.to_csv('edges.csv', index=False)